<a href="https://colab.research.google.com/github/janeg1/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-03-03 02:22:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.24MB/s    in 0.2s    

2023-03-03 02:22:07 (6.24 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [16]:
total_votes_df = df.select(["product_id","helpful_votes","total_votes","vine"])
new_total_votes_df= total_votes_df[total_votes_df.total_votes >=20.0]
new_total_votes_df.show(5)


+----------+-------------+-----------+----+
|product_id|helpful_votes|total_votes|vine|
+----------+-------------+-----------+----+
|B00ZI3R5VG|           19|         20|   N|
|B003DZ7VOW|           19|         20|   N|
|B00I1LM5SK|           30|         30|   N|
|B002ZLE41I|            8|         30|   N|
|B00OAY5ZPC|           35|         37|   N|
+----------+-------------+-----------+----+
only showing top 5 rows



In [17]:
#DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
#next_df= new_total_votes_df(["product_id", "votes"()])
#next_df.show(5)
next_df=new_total_votes_df.filter(new_total_votes_df["helpful_votes"]/new_total_votes_df["total_votes"] >=0.5)
next_df.show(5)

+----------+-------------+-----------+----+
|product_id|helpful_votes|total_votes|vine|
+----------+-------------+-----------+----+
|B00ZI3R5VG|           19|         20|   N|
|B003DZ7VOW|           19|         20|   N|
|B00I1LM5SK|           30|         30|   N|
|B00OAY5ZPC|           35|         37|   N|
|B000JQJS6M|           19|         22|   N|
+----------+-------------+-----------+----+
only showing top 5 rows



In [26]:
vineY_df=next_df.filter (next_df["vine"]== "Y")
vineY_df.show()

+----------+-------------+-----------+----+
|product_id|helpful_votes|total_votes|vine|
+----------+-------------+-----------+----+
|B00V0G9OGE|           49|         52|   Y|
|B00M1Y5AIU|           34|         39|   Y|
|B008AXURAW|           37|         39|   Y|
|B00IOOB1RU|           18|         25|   Y|
|B00ATUKPRE|           44|         45|   Y|
|B00ATUKPRE|           26|         27|   Y|
|B00B2GZI1M|           43|         48|   Y|
|B00B2GZJXE|           20|         23|   Y|
|B00ATUKQG4|           27|         30|   Y|
|B00B2GZJXE|           26|         28|   Y|
|B00791R1MI|          349|        370|   Y|
|B00791R1MI|          107|        121|   Y|
|B00AELHD0U|           22|         30|   Y|
|B00AELHD0U|           54|         60|   Y|
|B00791R1MI|          135|        144|   Y|
|B00ATULBA4|           20|         21|   Y|
|B00ATUL9TW|           13|         22|   Y|
|B0096IQZ8S|           61|         90|   Y|
|B0089K8CE4|           15|         20|   Y|
|B0089K8CHG|           24|      

In [38]:
vineN_df=next_df.filter (next_df["vine"]== "N")
vineN_df.show()

+----------+-------------+-----------+----+
|product_id|helpful_votes|total_votes|vine|
+----------+-------------+-----------+----+
|B00ZI3R5VG|           19|         20|   N|
|B003DZ7VOW|           19|         20|   N|
|B00I1LM5SK|           30|         30|   N|
|B00OAY5ZPC|           35|         37|   N|
|B000JQJS6M|           19|         22|   N|
|B00EX1GLXM|           27|         28|   N|
|B00WUY9RGM|           25|         26|   N|
|B00U0XC1T0|           14|         20|   N|
|B00YTYH16Y|           25|         27|   N|
|B00J5QR062|           26|         28|   N|
|B009XGOP8A|           48|         49|   N|
|B00BKQTA4A|           19|         20|   N|
|B013XHV4GU|           27|         28|   N|
|B00YTY8MQW|           24|         24|   N|
|B000B55AEK|           21|         21|   N|
|B00TU78EXE|           33|         46|   N|
|B00NIF9H3U|           19|         24|   N|
|B00K92JHX0|           17|         20|   N|
|B006ZMHUP0|           25|         26|   N|
|B012UTRAFG|           23|      

In [27]:
# Total number of reviews "paid" Vine program
vineY_df.count()

47

In [28]:
# Total number of reviews "not paid" Vine program
vineN_df.count()

8362

In [31]:
from pyspark.sql.functions import count

In [44]:
#Get total number of reviews for Vine program
next_df.select(count(df.vine)).show()

+-----------+
|count(vine)|
+-----------+
|       8409|
+-----------+



In [45]:
# Total number of 5 star reviews for Vine program
fivestar_df=next_df.filter (df["star_rating"]==5)
fivestar_df.count()

4347

In [46]:
import pyspark.sql.functions as F

In [50]:
fivestar_df.groupby("vine").agg(
    (F.count("vine")).alias("count"),
    (F.count("vine")/fivestar_df.count()).alias("percentage")
).show()

+----+-----+--------------------+
|vine|count|          percentage|
+----+-----+--------------------+
|   Y|   15|0.003450655624568668|
|   N| 4332|  0.9965493443754313|
+----+-----+--------------------+

